# IT Incident SLA Comliance Analysis

Service Level Agreement (SLA) compliance represents a critical performance metric in IT service management, directly impacting customer satisfaction, operational efficiency, and business continuity. This analysis examines cleaned IT incident data to identify systematic patterns affecting SLA compliance rates and builds predictive models to forecast potential SLA breaches.

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from collections import Counter
import warnings
import shap
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns',
              None)  # Display all columns in DataFrame output.
pd.set_option('display.max_rows',
              None)  # Display all rows in DataFrame output.
# Load data from dataset
df = pd.read_csv('../data/incidents_cleaned.csv')

# Display DataFrame information
df.info()
print(f"\nDataset shape: {df.shape}")
print(f"\nTarget Variable Distribution:")
print(df['made_sla'].value_counts())
print(f"SLA Compliance Rate: {df['made_sla'].mean():.1%}")
print(f"SLA Breach Rate: {(1 - df['made_sla'].mean()):.1%}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6729 entries, 0 to 6728
Data columns (total 47 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   number                         6729 non-null   object 
 1   incident_state                 6729 non-null   object 
 2   active                         6729 non-null   bool   
 3   reassignment_count             6729 non-null   int64  
 4   reopen_count                   6729 non-null   int64  
 5   sys_mod_count                  6729 non-null   int64  
 6   made_sla                       6729 non-null   bool   
 7   caller_id                      6727 non-null   object 
 8   opened_by                      6432 non-null   object 
 9   opened_at                      6729 non-null   object 
 10  sys_created_by                 3784 non-null   object 
 11  sys_created_at                 3784 non-null   object 
 12  sys_updated_by                 6729 non-null   o

## 1. Feature Engineering

### 1.1 Operational Complexity Features

In [7]:
df_fe = df.copy()

Rationale: EDA revealed sys_mod_count (r=-0.37) and reassignment_count (r=-0.25) as top predictors. Raw counts don't capture intensity (speed of handling), so we create ratio features normalized by incident duration.

In [8]:
# Convert hours to days for readability
# Add 0.01 to prevent division by zero for same-day incidents
df_fe['closed_time_days'] = df_fe['closed_time_hours'] / 24 + 0.01

# Intensity metrics: operations per day
# High values indicate rapid, chaotic handling (bad sign)
df_fe['mod_per_day'] = df_fe['sys_mod_count'] / df_fe['closed_time_days']
df_fe['reassign_per_day'] = df_fe['reassignment_count'] / df_fe[
    'closed_time_days']
df_fe['reopen_per_day'] = df_fe['reopen_count'] / df_fe['closed_time_days']

# Weighted activity score
# Weights based on EDA impact: modifications (1x), reassignments (2x), reopens (3x)
# Reopens weighted highest as they indicate failed resolution attempts
df_fe['activity_score'] = (df_fe['sys_mod_count'] +
                           df_fe['reassignment_count'] * 2 +
                           df_fe['reopen_count'] * 3)

# Binary complexity flags based on EDA-identified thresholds
# has_reassignment: EDA showed 0 reassignments = 93.1% SLA vs 1+ = 84.8% SLA (8.3% gap)
df_fe['has_reassignment'] = (df_fe['reassignment_count']
                          > 0).astype(int)  # 93.1% vs 84.8%

# high_modification: EDA showed 0-3 mods = >99% SLA vs 4+ mods = <95% SLA (threshold at 3)
df_fe['high_modification'] = (df_fe['sys_mod_count'] > 3).astype(int)  # >99% vs <95%

# is_complex: Combined threshold (>4 mods OR >2 reassignments)
# Captures incidents requiring extensive handling regardless of type
df_fe['is_complex'] = ((df_fe['sys_mod_count'] > 4) |
                       (df_fe['reassignment_count'] > 2)).astype(int)

print(f"  ✓ Created 7 complexity features")

print(f"  ✓ Created 7 operational complexity features:")
print(
    f"    - mod_per_day, reassign_per_day, reopen_per_day (intensity ratios)")
print(
    f"    - activity_score (weighted complexity: mods + 2×reassigns + 3×reopens)"
)
print(f"    - has_reassignment, high_modification, is_complex (binary flags)")

# Validate new feature
complex_sla = df_fe.groupby('is_complex')['made_sla'].mean()
print(f"\n  Validation - is_complex feature:")
print(f"    Simple incidents (0): {complex_sla[0]:.1%} SLA compliance")
print(f"    Complex incidents (1): {complex_sla[1]:.1%} SLA compliance")
print(
    f"    Performance gap: {(complex_sla[0]-complex_sla[1])*100:.1f} percentage points"
)

  ✓ Created 7 complexity features
  ✓ Created 7 operational complexity features:
    - mod_per_day, reassign_per_day, reopen_per_day (intensity ratios)
    - activity_score (weighted complexity: mods + 2×reassigns + 3×reopens)
    - has_reassignment, high_modification, is_complex (binary flags)

  Validation - is_complex feature:
    Simple incidents (0): 98.7% SLA compliance
    Complex incidents (1): 69.4% SLA compliance
    Performance gap: 29.2 percentage points


### 1.2 Severity Features

Rationale: Priority, Impact, and Urgency show strong multicollinearity (r=0.75-0.89), indicating they capture overlapping severity information. Using all three would introduce redundancy. We create binary "high severity" flags instead. EDA showed Priority has widest performance gap (Critical: 53.3% vs Low: 95.5% = 42.2% gap)From EDA: Priority shows 42.2% gap, but high multicollinearity with impact/urgency

In [10]:
# Binary severity flags (convert categorical to 0/1)
# 1 = High severity, 0 = Low severity
df_fe['is_high_priority'] = df_fe['priority'].isin(
    ['1 - Critical', '2 - High']).astype(int)
df_fe['is_high_impact'] = df_fe['impact'].isin(['1 - High']).astype(int)
df_fe['is_high_urgency'] = df_fe['urgency'].isin(['1 - High']).astype(int)

# Combined severity flag: ANY dimension is high
# Captures incidents marked as severe in at least one classification system
df_fe['is_high_severity'] = ((df_fe['is_high_priority'] == 1) |
                             (df_fe['is_high_impact'] == 1) |
                             (df_fe['is_high_urgency'] == 1)).astype(int)

print("✓ Created 4 severity-based features:")
print("  - is_high_priority: 1 if Critical/High, 0 otherwise")
print("  - is_high_impact: 1 if High impact, 0 otherwise")
print("  - is_high_urgency: 1 if High urgency, 0 otherwise")
print("  - is_high_severity: 1 if ANY dimension is high, 0 otherwise")

# Validate combined severity feature
sev_sla = df_fe.groupby('is_high_severity')['made_sla'].mean()
print(f"\nValidation - is_high_severity feature:")
print(f"  Low severity (0): {sev_sla[0]:.1%} SLA compliance")
print(f"  High severity (1): {sev_sla[1]:.1%} SLA compliance")
print(
    f"  Performance gap: {(sev_sla[0]-sev_sla[1])*100:.1f} percentage points")

✓ Created 4 severity-based features:
  - is_high_priority: 1 if Critical/High, 0 otherwise
  - is_high_impact: 1 if High impact, 0 otherwise
  - is_high_urgency: 1 if High urgency, 0 otherwise
  - is_high_severity: 1 if ANY dimension is high, 0 otherwise

Validation - is_high_severity feature:
  Low severity (0): 87.4% SLA compliance
  High severity (1): 54.9% SLA compliance
  Performance gap: 32.5 percentage points


### 1.3 Interaction Features

Rationale: Combinations of factors may have compounding effects on SLA compliance. High severity + High complexity likely exhibits multiplicative negative impact rather than simple additive effect.

In [12]:
# Critical interaction: High severity + High complexity
# Worst-case scenario: urgent issue requiring extensive handling
df_fe['severity_complexity'] = (df_fe['is_high_severity'] *
                                df_fe['is_complex']).astype(int)

# Priority confirmation + Complexity (from EDA: confirmation has 14.7% SLA gap)
# Confirmed complex cases represent highest-risk incidents
df_fe['confirmed_complex'] = (df_fe['u_priority_confirmation'] *
                              df_fe['is_complex']).astype(int)

# High priority + Reassignment
# Urgent cases that got bounced around (routing failures on critical issues)
df_fe['priority_reassign'] = (df_fe['is_high_priority'] *
                              df_fe['has_reassignment']).astype(int)

print("✓ Created 3 interaction features:")
print("  - severity_complexity: High severity AND complex incident")
print("  - confirmed_complex: Priority confirmed AND complex")
print("  - priority_reassign: High priority AND has reassignment")

# Validate most critical interaction
risk_sla = df_fe.groupby('severity_complexity')['made_sla'].mean()
print(f"\nValidation - severity_complexity feature:")
print(f"  No risk (0): {risk_sla[0]:.1%} SLA compliance")
print(f"  High risk (1): {risk_sla[1]:.1%} SLA compliance")
print(
    f"  Performance gap: {(risk_sla[0]-risk_sla[1])*100:.1f} percentage points"
)

print("\n" + "-" * 80)
print(f"Feature Engineering Summary:")
print(f"  Original features: {df.shape[1]}")
print(f"  New engineered features: {df_fe.shape[1] - df.shape[1]}")
print(f"  Total features after engineering: {df_fe.shape[1]}")
print("-" * 80)

✓ Created 3 interaction features:
  - severity_complexity: High severity AND complex incident
  - confirmed_complex: Priority confirmed AND complex
  - priority_reassign: High priority AND has reassignment

Validation - severity_complexity feature:
  No risk (0): 87.3% SLA compliance
  High risk (1): 50.3% SLA compliance
  Performance gap: 36.9 percentage points

--------------------------------------------------------------------------------
Feature Engineering Summary:
  Original features: 51
  New engineered features: 11
  Total features after engineering: 62
--------------------------------------------------------------------------------


### 1.4 Feature Encoding

#### 1.4.1 Ordinal Encoding

Priority, Impact, and Urgency have natural ordering from low to high severity. Preserve this ordering with numeric encoding

In [13]:
df_enc = df_fe.copy()

In [14]:
# Priority (1=Low, 4=Critical)
priority_map = {
    '4 - Low': 1,
    '3 - Moderate': 2,
    '2 - High': 3,
    '1 - Critical': 4
}
df_enc['priority_enc'] = df_enc['priority'].map(priority_map)

# Impact (1=Low, 3=High)
impact_map = {'3 - Low': 1, '2 - Medium': 2, '1 - High': 3}
df_enc['impact_enc'] = df_enc['impact'].map(impact_map)

# Urgency (1=Low, 3=High)
urgency_map = {'3 - Low': 1, '2 - Medium': 2, '1 - High': 3}
df_enc['urgency_enc'] = df_enc['urgency'].map(urgency_map)

print("✓ Ordinal encoded 3 variables:")
print("  - priority_enc: 1 (Low) → 4 (Critical)")
print("  - impact_enc: 1 (Low) → 3 (High)")
print("  - urgency_enc: 1 (Low) → 3 (High)")


✓ Ordinal encoded 3 variables:
  - priority_enc: 1 (Low) → 4 (Critical)
  - impact_enc: 1 (Low) → 3 (High)
  - urgency_enc: 1 (Low) → 3 (High)


#### 1.4.2 Label Encoding

assignment_group and category have many unique values without natural ordering

In [18]:
# assignment_group (from EDA: 56% performance variance across groups)
le_group = LabelEncoder()
df_enc['assignment_group'] = df_enc['assignment_group'].fillna('Unknown')
df_enc['assignment_group_enc'] = le_group.fit_transform(
    df_enc['assignment_group'])

# category (from EDA: 21.1% performance gap)
le_cat = LabelEncoder()
df_enc['category'] = df_enc['category'].fillna('Unknown')
df_enc['category_enc'] = le_cat.fit_transform(df_enc['category'])

print(f"✓ Label encoded 2 variables:")
print(f"  - assignment_group_enc: {len(le_group.classes_)} unique groups")
print(f"  - category_enc: {len(le_cat.classes_)} unique categories")

✓ Label encoded 2 variables:
  - assignment_group_enc: 65 unique groups
  - category_enc: 47 unique categories


#### 1.4.3 Binary Encoding

Binary features created during feature engineering need to be explicitly typed as integers

In [19]:
binary_vars = [
    'knowledge', 'u_priority_confirmation', 'made_sla', 'has_reassignment',
    'high_modification', 'is_complex', 'is_high_priority', 'is_high_impact',
    'is_high_urgency', 'is_high_severity', 'severity_complexity',
    'confirmed_complex', 'priority_reassign'
]

for var in binary_vars:
    if var in df_enc.columns:
        df_enc[var] = df_enc[var].astype(int)

verified_count = len([v for v in binary_vars if v in df_enc.columns])
print(f"✓ Verified {verified_count} binary variables as integer type")

✓ Verified 13 binary variables as integer type
